In [1]:
import torch
from torchvision import transforms
# from test_tube import Experiment
from torchvision import transforms

In [3]:
import argparse, os, json, sys
sys.path.insert(0,'/home/niell_lab/Documents/github/FreelyMovingEphys/')
from glob import glob
from tqdm import tqdm
from project_analysis.inception_loop.base import CorePlusReadout2d
from project_analysis.inception_loop.datasets import WorldcamDataset3D
from torch.utils.data import Dataset, DataLoader, Subset
from project_analysis.inception_loop.core import Stacked2dCore
import torch.nn as nn
import yaml

In [5]:
n = -1
version = 0
modeltype = 'inceptionloop' 
filename = '/home/niell_lab/data/freely_moving_ephys/inception_loop/inceptionloop/version_2/config.yaml'
ckpt_path = glob('/home/niell_lab/data/freely_moving_ephys/inception_loop/inceptionloop/version_2/checkpoints/*.pt')[n]
print(ckpt_path)
with open(filename, 'r') as file:
    try:
        args = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)
print(args)

/home/niell_lab/data/freely_moving_ephys/inception_loop/inceptionloop/version_2/checkpoints/spike_pred_epoch0.pt
{'batch_size': 1152, 'csv': '/home/niell_lab/data/freely_moving_ephys/inception_loop/inputs/train_history_dropnans.csv', 'hidden_channels': 2, 'hidden_kern': 27, 'history_size': 0, 'input_channels': 3, 'input_kern': 7, 'learning_rate': 1e-05, 'root_dir': '/home/niell_lab/data/freely_moving_ephys/inception_loop/', 'split_frac': 0.8}


In [7]:
SetRange = transforms.Lambda(lambda X: 2 * X - 1.)
transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                            transforms.Resize((args['img_resize'],args['img_resize'])),
                            transforms.ToTensor(),
                            SetRange])

dataset = WorldcamDataset3D(args['csv'], args['history_size'], input_path, transform=transform)

num_units = len(dataset.metadata['SR0'])

startind = dataset.metadata.index[0]; endind = dataset.metadata.index[-1]; splitind = int((endind - startind) * args['split_frac'])

training_data = Subset(dataset,torch.arange(startind,splitind))
testing_data = Subset(dataset,torch.arange(splitind,endind))

train_dataloader = DataLoader(training_data, batch_size=args['batch_size'], num_workers=args['num_workers'], persistent_workers=True, pin_memory=True, prefetch_factor=5)
test_dataloader = DataLoader(testing_data, batch_size=args['batch_size'], num_workers=args['num_workers'])

model = Stacked3dCore(args['input_channels'], args['hidden_channels'], args['input_kern'], args['hidden_kern'], num_units, spike_history_len=args['history_size'], img_resize=args['img_resize'])
model.to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=args['learning_rate'])

KeyError: 'img_resize'